In [1]:
from SagasuSubs import utils,models,patch
from pathlib import Path

patch.patch()

In [2]:
workdir = Path('.') / 'subtitles' / 'subs_list'

files = [*utils.iterate_files(workdir,'.ass')]

files[:5]

[PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][01][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][02][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][03][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][04][480P][简体外挂].ass'),
 PosixPath('subtitles/subs_list/animation/1970/(1970.4.1)明日之丈 第1季/BD/冷番补完字幕组/[冷番补完][明日之丈][あしたのジョー][05][480P][简体外挂].ass')]

In [3]:
from tqdm.notebook import tqdm
from traceback import format_exc,print_exc
import re

from SagasuSubs.database import FileCrud,DialogCrud,dto

DB_PATH = Path('.') / 'db.sqlite3'
file_crud = FileCrud(DB_PATH)
dialog_crud =DialogCrud(DB_PATH)


try:
    file_crud.create_table()
    dialog_crud.create_table()
except Exception as e:
    print_exc(-1)

PATH_REGEX = re.compile(
    r'^.+/(?P<year>\d+?)/\((?P<date>[0-9.]+?)\)(?P<name>.+?)/(?P<child_path>.+)$'
)

        
    

Traceback (most recent call last):
  File "/home/mix/Documents/SagasuSubs/.venv/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 717, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: table files already exists

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/mix/Documents/SagasuSubs/.venv/lib/python3.9/site-packages/sqlalchemy/engine/default.py", line 717, in do_execute
    cursor.execute(statement, parameters)
sqlalchemy.exc.OperationalError: (sqlite3.OperationalError) table files already exists
[SQL: 
CREATE TABLE files (
	id INTEGER NOT NULL, 
	filename VARCHAR(1024) NOT NULL, 
	sha1 VARCHAR(40) NOT NULL, 
	path VARCHAR NOT NULL, 
	series_name VARCHAR, 
	series_id INTEGER, 
	episode_name VARCHAR, 
	episode_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(series_id) REFERENCES series (id), 
	FOREIGN KEY(episode_id) REFERENCES episode (id)
)

]
(Background on this error 

In [ ]:
progress = tqdm(files)

skipped = 0

for count,file in enumerate(progress):
    progress.set_description(f'{skipped=}, {skipped/(count+1)=:.3%}')
    try:
        data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
    except Exception:
        progress.write(
            '\n-->\t'.join(
                [f'Error occurred during processing file {file}:'] +
                format_exc(-3).splitlines()
            )
        )
        skipped += 1
        continue

    if not (matched := PATH_REGEX.match(str(file))):
        progress.write(f'{data.path=} does not have valid regex data, skip')
        skipped += 1
        continue

    if existed_file:=file_crud.read_by_sha1(data.sha1):
#         progress.write(f'{data.filename=} conflicted with {existed_file.id=}, {existed_file.filename=} in database, skip')
        skipped += 1
        continue
        
    file_model = file_crud.create(dto.FileCreate(
        filename=data.filename,
        sha1=data.sha1,
        path=str(data.path),
        series_name=matched.group('name'))
    )

    dialog_crud.create_bulk(
            [
                dto.DialogCreate(content=dialog.content,begin=dialog.begin,end=dialog.end,file_id=file_model.id)
                for dialog in data.dialogs
            ]
        )
        

  0%|          | 0/50231 [00:00<?, ?it/s]

/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='encoding',v='1Style: Karaoke0' due to ValueError("invalid literal for int() with base 10: '1Style: Karaoke0'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:00.-40' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:00.-40' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='marginv',v='' due to ValueError("invalid litera

Error occurred during processing file subtitles/subs_list/animation/2008/(2008.4.6)绝对可怜CHILDREN/BD/[アニメ BD]/[アニメ BD] 絶対可憐チルドレン 第27話 「宣戦布告！パンドラからの挑戦状」.ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_34135/4174801234.py", line 8, in <module>
-->	    data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
-->	  File "/home/mix/Documents/SagasuSubs/SagasuSubs/models.py", line 62, in from_file
-->	    DialogContent(
-->	  File "pydantic/main.py", line 406, in pydantic.main.BaseModel.__init__
-->	pydantic.error_wrappers.ValidationError: 2 validation errors for DialogContent
-->	begin
-->	  none is not an allowed value (type=type_error.none.not_allowed)
-->	end
-->	  none is not an allowed value (type=type_error.none.not_allowed)
Error occurred during processing file subtitles/subs_list/animation/2008/(2008.4.6)绝对可怜CHILDREN/BD/[アニメ BD]/[アニメ BD] 絶対可憐チルドレン 第28話 「羞月閉花！蕾見チュー意報」.ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_34135/4

/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:-10.-60' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:-10.-60' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:-6.-45' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:-5.-18' due to AttributeError("'NoneType' object ha

Error occurred during processing file subtitles/subs_list/animation/2008/(2008.4.6)绝对可怜CHILDREN/BD/[アニメ BD]/[アニメ BD] 絶対可憐チルドレン 第45話 「天賦自然！ギフト·オブ チルドレン」.ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_34135/4174801234.py", line 8, in <module>
-->	    data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
-->	  File "/home/mix/Documents/SagasuSubs/SagasuSubs/models.py", line 62, in from_file
-->	    DialogContent(
-->	  File "pydantic/main.py", line 406, in pydantic.main.BaseModel.__init__
-->	pydantic.error_wrappers.ValidationError: 2 validation errors for DialogContent
-->	begin
-->	  none is not an allowed value (type=type_error.none.not_allowed)
-->	end
-->	  none is not an allowed value (type=type_error.none.not_allowed)


...

Error occurred during processing file subtitles/subs_list/animation/2015/(2015.1.8)铳皇无尽的法夫纳/BD/枪皇无尽的法夫纳BDRip/[reinforce] juuou mujin no fafnir - 09 (bdrip 1920x1080 x264 flac).ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_34135/4174801234.py", line 8, in <module>
-->	    data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
-->	  File "/home/mix/Documents/SagasuSubs/SagasuSubs/models.py", line 62, in from_file
-->	    DialogContent(
-->	  File "pydantic/main.py", line 406, in pydantic.main.BaseModel.__init__
-->	pydantic.error_wrappers.ValidationError: 2 validation errors for DialogContent
-->	begin
-->	  none is not an allowed value (type=type_error.none.not_allowed)
-->	end
-->	  none is not an allowed value (type=type_error.none.not_allowed)
Error occurred during processing file subtitles/subs_list/animation/2015/(2015.1.8)铳皇无尽的法夫纳/BD/枪皇无尽的法夫纳BDRip/[reinforce] juuou mujin no fafnir - 10 (bdrip 1920x1080 x264 flac).ass:
-->	Traceback (mos

/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:-01.00' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:00.-98' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='start',v='0:00:-08.-98' due to AttributeError("'NoneType' object has no attribute 'groups'"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")
/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='end',v='0:00:-06.-98' due to AttributeError("'NoneType' object ha

Error occurred during processing file subtitles/subs_list/animation/2016/(2016.10.1)ViVid Strike!(S5)/BD/Sub [完][2016.10][ViVid Strike][01_12] NEW_NODE&amp;amp;Sakurato字幕 繁體 對應BDRIP[Rein/[ReinForce] Vivid Strike! 12 (BDRip 1920x1080 x264 FLAC).NEW_NODE&Sakurato.ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_34135/4174801234.py", line 8, in <module>
-->	    data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
-->	  File "/home/mix/Documents/SagasuSubs/SagasuSubs/models.py", line 62, in from_file
-->	    DialogContent(
-->	  File "pydantic/main.py", line 406, in pydantic.main.BaseModel.__init__
-->	pydantic.error_wrappers.ValidationError: 1 validation error for DialogContent
-->	begin
-->	  none is not an allowed value (type=type_error.none.not_allowed)


/home/mix/Documents/SagasuSubs/SagasuSubs/patch.py:59: UserWarning: Failed to parse field f='fontsize',v='' due to ValueError("could not convert string to float: ''"), fallback to None
  warn(f"Failed to parse field {f=!r},{v=!r} due to {e!r}, fallback to None")


Error occurred during processing file subtitles/subs_list/animation/2017/(2017.4.14)剑姬神圣谭/BD/字幕/Dungeon ni Deai o Motomeru no wa Machigatte Iru Darouka Gaiden Sword Oratoria - 07 (BD 1280x720 x264 AAC).ass:
-->	Traceback (most recent call last):
-->	  File "/tmp/ipykernel_34135/4174801234.py", line 8, in <module>
-->	    data = models.SubtitlePersist.from_file(file,format='ass',exclude_fx=True)
-->	  File "/home/mix/Documents/SagasuSubs/SagasuSubs/models.py", line 62, in from_file
-->	    DialogContent(
-->	  File "pydantic/main.py", line 406, in pydantic.main.BaseModel.__init__
-->	pydantic.error_wrappers.ValidationError: 2 validation errors for DialogContent
-->	begin
-->	  none is not an allowed value (type=type_error.none.not_allowed)
-->	end
-->	  none is not an allowed value (type=type_error.none.not_allowed)
Error occurred during processing file subtitles/subs_list/animation/2017/(2017.4.14)剑姬神圣谭/BD/字幕/Dungeon ni Deai o Motomeru no wa Machigatte Iru Darouka Gaiden Sword Oratoria